<a href="https://colab.research.google.com/github/jamiehadd/Math189AD-MathematicalDataScienceAndTopicModeling/blob/main/tutorials/Multiplicative_Updates_for_NMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Math 189: Multiplicative Updates for NMF

In this notebook, we'll compare an implementation of multiplicative updates to the built-in scikit-learn version on a toy dataset. (Check out this implementation vs. the one you created in class!)

##**Activity**

We’re going to try out [an implementation](https://github.com/jamiehadd/Math189AD-MathematicalDataScienceAndTopicModeling/blob/main/mult_ups.py) of multiplicative updates to train NMF.

### Tasks

* Visualize the NMF output of our implementation for various numbers of iterations.
* Compare the outputs of the built-in implementation to ours for various choices of iterations!

The dataset we'll explore is the Swimmer dataset, which was built by Donoho and Stodden for their NeurIPS 2004 paper [When does nonnegative matrix factorization give a correct decomposition into parts?](https://proceedings.neurips.cc/paper/2003/file/1843e35d41ccf6e63273495ba42df3c1-Paper.pdf)  This is a synthetic image dataset where the data points are images of a stick-figure model of a swimmer with four limbs that occur in various positions.

In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.decomposition import NMF
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

warnings.simplefilter("ignore")                                                 #the sklearn NMF function likes to warn when we hit max_iters, so we'll suppress warnings here

In [ ]:
url="https://raw.githubusercontent.com/jamiehadd/Math189AD-MathematicalDataScienceAndTopicModeling/main/mult_ups.py"
!wget --no-cache --backups=1 {url}
from mult_ups import mult_ups

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd '/content/drive/Shareddrives/Math 189AD FA22: Datasets'

In [ ]:
mat = scipy.io.loadmat("Swimmer.mat")                 #load the dataset
X = mat['X']                                          #in this .mat file, the data is called X
numPixels, numPics = X.shape                          #record the number of images in the dataset, and number of pixels in each image

pic17 = np.reshape(X[:,17],(11,20));                  #grab the 17th image and reshape into image format
pic170 = np.reshape(X[:,170],(11,20));                #grab the 170th image and reshape into image format

In [ ]:
plt.figure(figsize=[15,6])

plt.suptitle("Swimmer Images");
plt.subplot(1,2,1);
plt.imshow(pic17);                                   #display images
plt.title("Image 17");
plt.xticks([]);
plt.yticks([]);

plt.subplot(1,2,2);
plt.imshow(pic170);
plt.title("Image 170");
plt.xticks([]);
plt.yticks([]);

Now we'll apply our NMF implementation and the built-in version in scikit-learn and compare the results!

Let's start by running our implementation on the Swimmer dataset to produce a model of rank 10.  We'll visualize the NMF model error over the iterations of the method.  Note: this is **not** plotting error vs. model rank!

In [ ]:
M = 150
A, S, errs = mult_ups(X,10,M = M)

In [ ]:
plt.figure(figsize=[10,10])

plt.suptitle("NMF Errors throughout Multiplicative Updates Iterations");
plt.semilogy(range(1,M+2),errs);                                                  #plot the error
plt.xlabel("iteration, $n$");
plt.ylabel("model error, $\|X - AS\|_F^2$");

Let's visualize the basis images for this trial!

In [ ]:
plt.figure(figsize=[15,5])

plt.suptitle("NMF Basis Elements: Model Rank 10")
for i in range(10):
  plt.subplot(2,5,i+1)
  plt.imshow(np.reshape(A[:,i],(11,20)))
  plt.xticks([])
  plt.yticks([])

***Task 1:*** Compare these visualization of the outputs for various numbers of iterations!  Consider the error plot above when deciding which numbers of iterations to try.

In [ ]:
# FILL

***Task 2:*** Compare our implementation to the built-in implementation!

Getting the error values at each iteration for the built-in implementation is a bit of a pain, so I show you how to do it below!

In [ ]:
M = 150                                                                         #declare number of iterations
k = 10
A = np.abs(np.random.randn(np.shape(X)[0], k))
S = np.abs(np.random.randn(k, np.shape(X)[1]))
errs = [np.linalg.norm(X-A@S,'fro')**2];

for i in range(M):
    model = NMF(n_components=k, init='custom', random_state=0, max_iter = 1, solver = 'mu')    #learn the NMF model with rank k
    A = model.fit_transform(X,W=A,H=S)                                          #access the NMF factor matrices
    S = model.components_
    errs.append(np.linalg.norm(X-A@S,'fro')**2)

In [ ]:
plt.figure(figsize=[10,10])

plt.suptitle("NMF Errors throughout Multiplicative Updates Iterations");
plt.semilogy(range(1,M+2),errs);                                                  #plot the error
plt.xlabel("iteration, $n$");
plt.ylabel("model error, $\|X - AS\|_F^2$");